# Train and visualize a model in Tensorflow - Part 5: Using Tensorboard

Neural networks have been widely critized because of the lack of interpretation of their internal parameters. In this notebook we will present some techniques visualize the model structure and its behaviour during training.

The lack of interpretability leads, among other thigns, to make neural models error prone. While this is true, we still have some tools to try to debug our network and to understand what the model is doing.

Tensorflow comes with its own (and very complete) visualization tool: Tensorboard. In the rest of this tutorial, we will explain how to use Tensorboard to inpect the progress of the network while training, check the execution graph and compare different models.

As first step, we use the same code as in previous notebooks to load the dataset.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Load the dataset into a numpy keyed structure
newsgroups = np.load('./resources/newsgroup.npz')

# Define the batch size
batch_size = 100

def dataset_input_fn(dataset):
    """
    Creates an input function using the `numpy_input_fn` method from
    tensorflow, based on the dataset we want to use.
    
    Args:
        dataset: String that represents the dataset
        (should be `train` or `test`)
    
    Returns:
        An `numpy_input_fn` function to feed to an estimator
    """
    assert dataset in ('train', 'test'),\
        "The selected dataset should be `train` or `test`"
    
    return tf.estimator.inputs.numpy_input_fn(
        # A dictionary of numpy arrays that match each array with the
        # corresponding column in the model. For this case we only
        # have "one" colum which represents the whole array.
        x={'input_data': newsgroups['%s_data' % dataset]},
        # The target array
        y=newsgroups['%s_target' % dataset],
        # The batch size to iterate the data in small fractions
        batch_size=batch_size,
        # If the dataset is `test` only run once
        num_epochs=1 if dataset == 'test' else None,
        # Only shuffle the dataset for the `train` data
        shuffle=dataset == 'train'
    )

In [3]:
input_size = newsgroups['train_data'].shape[1]
num_classes = newsgroups['labels'].shape[0]

feature_columns = [tf.feature_column.numeric_column(
    'input_data', shape=(input_size,))]

## Visualizing the execution graph

For the first experiment, we define the same model as before.

In [15]:
big_model = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=(5000, 2000,),
    n_classes=num_classes,
    model_dir="models/bigDNN")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_session_config': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_service': None, '_save_checkpoints_steps': None, '_is_chief': True, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f939ba74358>, '_keep_checkpoint_max': 5, '_model_dir': 'models/bigDNN', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600}


After creating the DNNClassifier object, we can already lunch tensoboard and explore the graph. To start Tensorboard, in your terminal go to the folder where you downloaded the tutorial and type:

```$ tensorboard --dirname models```

## Visualize learning

Now, we would like to inspect the performance of the classifier while it is training. In the contrib.learn module of tensorflow we found an Experiment class that will run the train and evaluation cycle for us. Even if this not hard to implement using a for loop, we recommend to use the functions provided by tensorflow as they support training on multiple servers.

In the following cell, we indicate that we want to train our classifier over 2000 epocs, and evaluate its performance in the test dataset every 100 iterations

In [16]:
big_experiment = tf.contrib.learn.Experiment(
    big_model,
    train_input_fn=dataset_input_fn("train"),
    eval_input_fn=dataset_input_fn("test"),
    train_steps=2000,
    train_steps_per_iteration=100
)
big_experiment.continuous_train_and_eval()

INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/bigDNN/model.ckpt.
INFO:tensorflow:step = 1, loss = 299.495
INFO:tensorflow:Saving checkpoints for 100 into models/bigDNN/model.ckpt.
INFO:tensorflow:Loss for final step: 72.389.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-16-21:09:06
INFO:tensorflow:Restoring parameters from models/bigDNN/model.ckpt-100
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:

INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:10:47
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.886883, average_loss = 0.378348, global_step = 400, loss = 37.4962
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/bigDNN/model.ckpt-400
INFO:tensorflow:Saving checkpoints for 401 into models/bigDNN/model.ckpt.
INFO:tensorflow:step = 401, loss = 2.07265
INFO:tensorflow:Saving checkpoints for 500 into models/bigDNN/model.ckpt.
INFO:tensorflow:Loss for final step: 4.2747.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-16-21:11:17
INFO:tensorflow:Restoring parameters from models/bigDNN/model.ckpt-500
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]

INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:12:57
INFO:tensorflow:Saving dict for global step 800: accuracy = 0.893521, average_loss = 0.4148, global_step = 800, loss = 41.1088
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/bigDNN/model.ckpt-800
INFO:tensorflow:Saving checkpoints for 801 into models/bigDNN/model.ckpt.
INFO:tensorflow:step = 801, loss = 0.519953
INFO:tensorflow:Saving checkpoints for 900 into models/bigDNN/model.ckpt.
INFO:tensorflow:Loss for final step: 0.111203.
INFO:tensorflow:Eva

INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:15:06
INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.891397, average_loss = 0.445988, global_step = 1200, loss = 44.1998
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/bigDNN/model.ckpt-1200
I

INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:17:17
INFO:

INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/10

({'accuracy': 0.89352095,
  'average_loss': 0.44748718,
  'global_step': 2000,
  'loss': 44.348335},
 [])

The DNNClassifier will automatically record the accuracy of the model, along with the loss. We can see the progress of these metrics in Tensorboard while the model is training, and eventually stop the training if the model starts to overfit.

## Comparing experiments

Now, what if we want to compare the performance with another model?

We can define a smaller model and see the same metrics as before in the same graph.

In [21]:
little_model = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=(500, 100,),
    n_classes=num_classes,
    model_dir="models/littleDNN")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_session_config': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_service': None, '_save_checkpoints_steps': None, '_is_chief': True, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f939b1a0080>, '_keep_checkpoint_max': 5, '_model_dir': 'models/littleDNN', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600}


In [22]:
little_experiment = tf.contrib.learn.Experiment(
    little_model,
    train_input_fn=dataset_input_fn("train"),
    eval_input_fn=dataset_input_fn("test"),
    train_steps=2000,
    train_steps_per_iteration=100
)
little_experiment.continuous_train_and_eval()

INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/littleDNN/model.ckpt.
INFO:tensorflow:step = 1, loss = 299.566
INFO:tensorflow:Saving checkpoints for 100 into models/littleDNN/model.ckpt.
INFO:tensorflow:Loss for final step: 56.1963.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-16-21:27:04
INFO:tensorflow:Restoring parameters from models/littleDNN/model.ckpt-100
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:t

INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:27:21
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.894583, average_loss = 0.364217, global_step = 400, loss = 36.0958
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/littleDNN/model.ckpt-400
INFO:tensorflow:Saving checkpoints for 401 into models/littleDNN/model.ckpt.
INFO:tensorflow:step = 401, loss = 2.25545
INFO:tensorflow:Saving checkpoints for 500 into models/littleDNN/model.ckpt.
INFO:tensorflow:Loss for final step: 1.00817.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-16-21:27:26
INFO:tensorflow:Restoring parameters from models/littleDNN/model.ckpt-500
INFO:tensorflow:Eva

INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:27:43
INFO:tensorflow:Saving dict for global step 800: accuracy = 0.89538, average_loss = 0.374744, global_step = 800, loss = 37.1391
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/littleDNN/model.ckpt-800
INFO:tensorflow:Saving checkpoints for 801 into models/littleDNN/model.ckpt.
INFO:tensorflow:step = 801, loss = 0.283583
INFO:tensorflow:Saving ch

INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-16-21:28:05
INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.898035, average_loss = 0.386132, global_step = 120

INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/

INFO:tensorflow:Restoring parameters from models/littleDNN/model.ckpt-2000
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100

({'accuracy': 0.89697289,
  'average_loss': 0.41810048,
  'global_step': 2000,
  'loss': 41.435959},
 [])

And finally we can compare to a model with a single layer.

In [24]:
single_layer_model = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=(1000,),
    n_classes=num_classes,
    model_dir="models/single_layer_NN")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_session_config': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_service': None, '_save_checkpoints_steps': None, '_is_chief': True, '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f939b63dc88>, '_keep_checkpoint_max': 5, '_model_dir': 'models/single_layer_NN', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600}


In [25]:
single_layer_experiment = tf.contrib.learn.Experiment(
    single_layer_model,
    train_input_fn=dataset_input_fn("train"),
    eval_input_fn=dataset_input_fn("test"),
    train_steps=2000,
    train_steps_per_iteration=100
)
single_layer_experiment.continuous_train_and_eval()

INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:step = 1, loss = 299.513
INFO:tensorflow:Saving checkpoints for 100 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:Loss for final step: 44.5894.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-17-22:14:07
INFO:tensorflow:Restoring parameters from models/single_layer_NN/model.ckpt-100
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluati

INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-17-22:14:36
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.910515, average_loss = 0.303765, global_step = 400, loss = 30.1047
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/single_layer_NN/model.ckpt-400
INFO:tensorflow:Saving checkpoints for 401 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:step = 401, loss = 4.69084
INFO:tensorflow:Saving checkpoints for 500 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:Loss for final step: 5.89316.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-17-22:14:45
INF

INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Finished evaluation at 2018-02-17-22:15:13
INFO:tensorflow:Saving dict for global step 800: accuracy = 0.914233, average_loss = 0.297888, global_step = 800, loss = 29.5222
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters fr

INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [3

INFO:tensorflow:Starting evaluation at 2018-02-17-22:16:26
INFO:tensorflow:Restoring parameters from models/single_layer_NN/model.ckpt-1600
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/single_layer_NN/model.ckpt-1900
INFO:tensorflow:Saving checkpoints for 1901 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:step = 1901, loss = 0.196552
INFO:tensorflow:Saving checkpoints for 2000 into models/single_layer_NN/model.ckpt.
INFO:tensorflow:Loss for final step: 0.367257.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2018-02-17-22:17:03
INFO:tensorflow:Restoring parameters from models/single_layer_NN/model.ckpt-2000
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflo

({'accuracy': 0.91210836,
  'average_loss': 0.31448451,
  'global_step': 2000,
  'loss': 31.167068},
 [])

Now go to Tensoboard and analyze how the different models behaved, comparing final performance and training time.